In [1]:
import gym, torch, random, copy, time
import matplotlib.pyplot as plt

In [10]:
class DQNAgent():
    
    def __init__(self, env, hidden_layer = 50, batch_size = 128, lr = 0.005, e=0.2, gamma=0.9):
        self.env = env
        layer_1 = torch.nn.Linear(env.observation_space.shape[0], hidden_layer)
        layer_2 = torch.nn.Linear(hidden_layer, env.action_space.n)
        layer_1.weight.data.fill_(0)
        layer_2.weight.data.fill_(0)

        self.model = torch.nn.Sequential(
            layer_1,
            torch.nn.ReLU(),
            layer_2
        )
        self.loss_fn = torch.nn.MSELoss(reduction="sum")
        self.optim = torch.optim.Adam(self.model.parameters(), lr=lr)
        
        self.mem = []
        self.batch_size = batch_size
        
        self.e = e
        self.gamma = gamma

    def save_transition(self, state, reward, action, done, next_state):
        self.mem.append((state, reward, action, done, next_state))
        
    def train(self):
        if len(self.mem)>=self.batch_size:
            while len(self.mem)!=0:
                state, reward, action, done, next_state = self.mem.pop()
                
                y_pred = self.model(torch.FloatTensor(state))
                
                y_target = y_pred.tolist()
                if done:
                    y_target[action] = reward
                else:
                    y_target[action] = reward+self.gamma*torch.max(self.model(torch.FloatTensor(next_state))).item()
                y_target_tensor = torch.FloatTensor(y_target)
                
                loss = self.loss_fn(y_pred, y_target_tensor)

                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
    
    def act(self, state):
        state = torch.FloatTensor(state)
        if random.random()<self.e:
            return self.env.action_space.sample()
        return torch.argmax(self.model(state)).item()

In [12]:
start_time = time.time()
env = gym.make('CartPole-v0')
gamma = 0.9
epsilon = 0.2
reward_arr = []
cum_reward = 0
num_episodes = 99999
hidden_layer = 50
batch_size = 128
lr = 0.01

reward_arr = []

#dqn = DQNAgent(env, lr=lr)

for _ in range(num_episodes):
    
    current_state = env.reset()
    episode_reward = 0
    done = False
    
    while not done:
        action = dqn.act(current_state)
        env.render()
        next_state, reward, done, info = env.step(action)
        
        dqn.save_transition(current_state, reward, action, done, next_state)
        dqn.train()
        
        episode_reward += reward
        cum_reward+=reward
        
    reward_arr.append(episode_reward)
        
    if _%100==0:
        print(_, cum_reward/(_+1))
env.close()
plt.plot(reward_arr)
print(time.time()-start_time)

0 13.0
100 10.594059405940595


KeyboardInterrupt: 

In [ ]:
plt.plot(reward_arr)
plt.show()

running_avg = []
val = 0

for i in range(len(reward_arr)):
    val += reward_arr[i]
    running_avg.append(val/(i+1))
plt.plot(running_avg)
plt.show()